# HDF5 Files

In [1]:
from glob import glob
import os
import re

import geopandas as gpd
import pandas as pd
import xarray as xr

data_path = os.path.join('data', '02-smap-hdf5-xarray')

In [2]:
!./bin/02-smap-hdf5-xarray-to-html.sh

[NbConvertApp] Converting notebook 02-smap-hdf5-xarray.ipynb to html
[NbConvertApp] Writing 298446 bytes to html/202205231526-smap-hdf5-xarray.html


In [3]:
boulder_county_path = os.path.join(
    data_path, 
    'boulder-county-boundary', 
    'County_Boundary', 
    'County_Boundary.shp')
boulder_county = gpd.read_file(boulder_county_path)
boulder_county.total_bounds

DriverError: data/02-smap-hdf5-xarray/boulder-county-boundary/County_Boundary/County_Boundary.shp: No such file or directory

In [5]:
smap_path_pattern = os.path.join(
    data_path, 'boulder-smap', 'granules', '*', '*.he5')
smap_path_list = glob(smap_path_pattern)
smap_file_df = pd.DataFrame({'path': smap_path_list})

In [12]:
smap_file_pattern = (
    r'SMAP_L4_SM_aup_(?P<date>\d{8})T(?P<time>\d{6})_Vv6032_001_HEGOUT.he5')
smap_file_re = re.compile(smap_file_pattern)
#smap_file_df[['date', 'time']]
smap_file_df.apply(
    lambda row: re.search(smap_path_re, row.path).groups(), 
    axis=1,
    result_type='expand')

,path


In [97]:
import rasterio
import rioxarray as rxr
with rasterio.open(smap_file_df.iloc[0].path) as src:
    subdatasets = src.subdatasets
    for sd in subdatasets:
        print(sd)

HDF5:data/02-smap-hdf5-xarray/boulder-smap/granules/227555905/SMAP_L4_SM_aup_20151116T150000_Vv6032_001_HEGOUT.he5://HDFEOS/GRIDS/Analysis_Data/Data_Fields/sm_rootzone_analysis
HDF5:data/02-smap-hdf5-xarray/boulder-smap/granules/227555905/SMAP_L4_SM_aup_20151116T150000_Vv6032_001_HEGOUT.he5://HDFEOS/GRIDS/Analysis_Data/Data_Fields/sm_surface_analysis
HDF5:data/02-smap-hdf5-xarray/boulder-smap/granules/227555905/SMAP_L4_SM_aup_20151116T150000_Vv6032_001_HEGOUT.he5://HDFEOS/GRIDS/FileMainGroup/Data_Fields/cell_lat
HDF5:data/02-smap-hdf5-xarray/boulder-smap/granules/227555905/SMAP_L4_SM_aup_20151116T150000_Vv6032_001_HEGOUT.he5://HDFEOS/GRIDS/FileMainGroup/Data_Fields/cell_lon


In [150]:
varname = 'HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis'
example = 'data/02-smap-hdf5-xarray/boulder-smap/granules/227555905/SMAP_L4_SM_aup_20151116T150000_Vv6032_001_HEGOUT.he5'
groupname = 'HFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis'
rxr.open_rasterio(example).squeeze()[varname]

/Users/elsa/opt/miniconda3/envs/ea-h5py/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/Users/elsa/opt/miniconda3/envs/ea-h5py/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/Users/elsa/opt/miniconda3/envs/ea-h5py/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/Users/elsa/opt/miniconda3/envs/ea-h5py/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = Datase

<xarray.DataArray 'HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis' (
                                                                                y: 4,
                                                                                x: 7)>
array([[0.154197, 0.097038, 0.088493, 0.083183, 0.085167, 0.147728, 0.152977],
       [0.178733, 0.105919, 0.090629, 0.084098, 0.089012, 0.150535, 0.171043],
       [0.182396, 0.106041, 0.092949, 0.083397, 0.081047, 0.097221, 0.101768],
       [0.138145, 0.101463, 0.085045, 0.077476, 0.089622, 0.102592, 0.227195]],
      dtype=float32)
Coordinates:
  * y            (y) float64 0.5 1.5 2.5 3.5
  * x            (x) float64 0.5 1.5 2.5 3.5 4.5 5.5 6.5
    band         int64 1
    spatial_ref  int64 0
Attributes:
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis_coordinates:  ...
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis_DIMENSION_LABE...
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis_fmissing_value...
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis_grid_mapping: ...
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis_long_name:    ...
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis_missing_value:...
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis_pixel_size:   ...
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis_pixel_units:  ...
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis_units:        ...
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis_valid_max:    ...
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis_valid_min:    ...
    HDFEOS_GRIDS_Analysis_Data_Data_Fields_sm_surface_analysis__FillValue:   ...
    _FillValue:                                                              ...
    scale_factor:                                                            ...
    add_offset:                                                              ...

In [134]:
import rasterio as rio
with rio.open(example) as groups:
    for name in groups.subdatasets:
        print(name)

HDF5:data/02-smap-hdf5-xarray/boulder-smap/granules/227555905/SMAP_L4_SM_aup_20151116T150000_Vv6032_001_HEGOUT.he5://HDFEOS/GRIDS/Analysis_Data/Data_Fields/sm_rootzone_analysis
HDF5:data/02-smap-hdf5-xarray/boulder-smap/granules/227555905/SMAP_L4_SM_aup_20151116T150000_Vv6032_001_HEGOUT.he5://HDFEOS/GRIDS/Analysis_Data/Data_Fields/sm_surface_analysis
HDF5:data/02-smap-hdf5-xarray/boulder-smap/granules/227555905/SMAP_L4_SM_aup_20151116T150000_Vv6032_001_HEGOUT.he5://HDFEOS/GRIDS/FileMainGroup/Data_Fields/cell_lat
HDF5:data/02-smap-hdf5-xarray/boulder-smap/granules/227555905/SMAP_L4_SM_aup_20151116T150000_Vv6032_001_HEGOUT.he5://HDFEOS/GRIDS/FileMainGroup/Data_Fields/cell_lon


/Users/elsa/opt/miniconda3/envs/ea-h5py/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [133]:
with open(example, 'rb') as h5file:
    h5ds = xr.open_dataset(h5file, engine='h5netcdf').load()
h5ds.groups

AttributeError: 'Dataset' object has no attribute 'groups'